In [ ]:
# !pip install telebot==0.0.5

In [ ]:
# !pip install pandas
# !pip install numpy
# !pip install schedule

In [2]:
# import pandas as pd
# import numpy as np
# import telebot
# import time
# import telebot
# import schedule
# import time
# from datetime import datetime
# import threading

In [20]:
import telebot
import schedule
import time
import threading
from datetime import datetime

# Создаем экземпляр бота (замените 'YOUR_BOT_TOKEN' на токен от BotFather)
bot = telebot.TeleBot('7658398564:AAEHMN2DH3odfLSoxiq6oZtkHga8z1MfX7E')

# Данные из таблицы в виде списка словарей
users_data = [
    {
        "chat_id": 850149704,
        "dz_bool": 0,
        "send_time": "07.03.2025, 16:28",
        "name": "Александр",
        "message_sent": False  # Флаг для отслеживания отправки сообщения
    },
    {
        "chat_id": 426803001,
        "dz_bool": 1,
        "send_time": "07.03.2025, 16:28",
        "name": "Влад",
        "message_sent": False
    },
    {
        "chat_id": 1078826317,
        "dz_bool": 0,
        "send_time": "07.03.2025, 16:26",
        "name": "Руслана",
        "message_sent": False
    }
]

# Блокировка для синхронизации доступа к users_data
users_data_lock = threading.Lock()

# Функция для отправки сообщения
def send_message_to_user(user):
    chat_id = user["chat_id"]
    name = user["name"]
    dz_bool = user["dz_bool"]
    
    # Формируем текст сообщения в зависимости от dz_bool
    if dz_bool == 1:
        message_text = f"Поздравляю, {name}, ты вошёл в 30% выполневших домашнее задание студентов нашего курса!"
    else:
        message_text = f"{name}, не забудьте отправить домашнее задание по теме: 'Я чемпион, я победитель, я TRUMP'."
    
    try:
        bot.send_message(chat_id, message_text)
        print(f"Сообщение отправлено пользователю {chat_id} ({name})")
        user["message_sent"] = True  # Помечаем сообщение как отправленное
    except telebot.apihelper.ApiTelegramException as e:
        print(f"Не удалось отправить сообщение пользователю {chat_id} ({name}): {e}")

# Функция для проверки и отправки сообщений
def check_and_send_messages():
    current_time = datetime.now().strftime("%d.%m.%Y, %H:%M")
    print(f"Проверка времени: {current_time}")
    
    # Блокируем доступ к users_data для других потоков
    with users_data_lock:
        for user in users_data:
            if user["send_time"] == current_time and not user["message_sent"]:
                send_message_to_user(user)

# Настройка расписания (проверка каждую минуту)
schedule.every(45).seconds.do(check_and_send_messages)

# Основной цикл для проверки расписания
def run_scheduler():
    while True:
        schedule.run_pending()
        time.sleep(30)  # Проверка каждые 30 секунд

# Запуск бота
if __name__ == '__main__':
    print('Бот запущен...')

    scheduler_thread = threading.Thread(target=run_scheduler)
    scheduler_thread.start()
    bot.polling(none_stop=True)
    schedule.clear()

Бот запущен...
